In [7]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import requests

Scraping NST Using BS4 and HTML Parsing

In [16]:
def timeProcessor(time,today):
    if time.endswith('ago'):
        return today.strftime('%Y-%m-%d')
    else:
       return datetime.strptime(time.split(' @ ')[0].strip(),'%b %d, %Y').strftime('%Y-%m-%d')

In [19]:
url = f"https://www.nst.com.my/news/politics?page="

In [20]:
page = -1
max_page = 944
total_info = []
total_articles = 0
today = datetime.today().date()

while page != max_page:
    page += 1
    driver = uc.Chrome()
    driver.get(f"https://www.nst.com.my/news/politics?page={page}")
    class_name_article = 'content.pl-2.pl-lg-3.col'
    WebDriverWait(driver,5).until(EC.visibility_of_element_located((By.CLASS_NAME,class_name_article)))
   
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    print(f'Printing {page}')   

    for article in soup.find_all('div',class_='content pl-2 pl-lg-3 col'):
        total_articles += 1
        title = article.find('h6',class_='field-title').text #get article titles
        description = article.find('div',class_='d-block article-teaser').text #get article descs
        category = article.find('span',class_='field-category').text
        time = timeProcessor(article.find('span',class_='created-ago').text,today)
        row_info = [title,description,time,category]
        total_info.append(row_info)  
    
    driver.close()

print(f'Total Articles : {total_articles}')
 

Printing 0
Printing 1
Printing 2
Printing 3
Printing 4


KeyboardInterrupt: 

In [21]:
total_info

[['Muda hopes for institutional reform with new cabinet lineup',
  'KUALA LUMPUR: With the announcement of the fresh Cabinet lineup, Malaysian United Democratic Alliance (Muda) expressed hope that the current lineup will implement institutional reform.',
  '2023-12-12',
  'Politics'],
 ["Ramkarpal's voluntary resignation to eliminate nepotism",
  "KUALA LUMPUR: Paving the way for his elder brother, Gobind Singh Deo, Ramkarpal Singh's voluntary move shows DAP's dedication to eliminating nepotism within the party. ",
  '2023-12-12',
  'Politics'],
 ["Pas veep: Anwar's new cabinet proves earlier promises were empty rhetoric",
  'KOTA BARU: Pas believes that the Prime Minister’s promises to reduce the size of the cabinet and expenses when he took over the government last year, was just empty rhetoric.',
  '2023-12-12',
  'Politics'],
 ["Pas issues ultimatum to party reps: Support Anwar and you're out",
  'KUALA LUMPUR: Pas representatives are warned that they will be dismissed if they prov

In [104]:
df = pd.DataFrame(total_info,columns=['title','description','publisheddate','section','site'])
df

,title,description,publisheddate,section,site
0,The $100bn ghost city in southern Malaysia,The Chinese-built city was supposed to be a th...,15 hours ago,News,Asia
1,Snoop Dogg & an ex-Wolves midfielder - meet Ma...,Former Wolves midfielder Hong Wan has moved fr...,2 days ago,Sport,Football
2,Forest City: Inside Malaysia's Chinese-built '...,"Built during the Chinese property boom, Forest...",6 days ago,News,Business
3,Malaysian killer released from Australian immi...,An ex-policeman convicted in a politically cha...,13 November,News,Australia
4,MotoGP: Enea Bastianini claims first race win ...,Ducati's Enea Bastianini claims his first race...,12 November,Sport,Motorsport
...,...,...,...,...,...
285,Najib Razak: Malaysian ex-PM gets 12-year jail...,Najib Razak gets 12 years in jail after he is ...,28 July 2020,News,Asia
286,Newsday: Guilty verdict for former Malaysian p...,"Live news, business and sport from around the ...",28 July 2020,Programmes,BBC World Service
287,Newsday: Former Malaysian PM guilty of abuse o...,"Live news, business and sport from around the ...",28 July 2020,Programmes,BBC World Service
288,North Korea v Malaysia: Asian Cup qualifier is...,"""Geo-political tension"" and ""safety and securi...",17 May 2017,Sport,Football


In [105]:
df.to_csv('BBCNews.csv',index=False)